## Beispiel-Project Regression 

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math
import statsmodels.api as sm

np.random.seed(42)  # Für reproduzierbare Ergebnisse

n_samples = 1000

# 1) Wohnfläche (square_meters): uniform zwischen 30 und 250
square_meters = np.random.uniform(30, 250, n_samples)

# 2) Anzahl Zimmer (num_rooms): ganzzahlig zwischen 1 und 10
num_rooms = np.random.randint(1, 11, n_samples)

# 3) Entfernung zum Zentrum (distance_to_center): uniform zw. 0 und 50 km
distance_to_center = np.random.uniform(0, 50, n_samples)

# 4) Baujahr (build_year): ganzzahlig zwischen 1950 und 2023
build_year = np.random.randint(1950, 2024, n_samples)

# 5) Zustand (condition_score): uniform zw. 0 und 1
condition_score = np.random.uniform(0, 1, n_samples)
# Normalverteiltes Rauschen mit Mittel=0, Standardabweichung=30000 (z.B.)
epsilon = np.random.normal(loc=0, scale=15000, size=n_samples)

house_price = (
    1500 * square_meters +
    8000 * num_rooms -
    3000 * np.log1p(distance_to_center) +  # log(1 + distance)
    15 * (build_year - 1950) +
    20000 * condition_score**2 +
    epsilon
)

df_houses = pd.DataFrame({
    'square_meters': square_meters,
    'num_rooms': num_rooms,
    'distance_to_center': distance_to_center,
    'build_year': build_year,
    'condition_score': condition_score,
    'house_price': house_price
})

n_outliers = 5
outlier_indices = np.random.choice(df_houses.index, n_outliers, replace=False)

# Extrem teure Immobilien z. B. + 500.000 bis + 1.000.000
df_houses.loc[outlier_indices, 'house_price'] += np.random.uniform(500000, 1000000, n_outliers)

df_houses.describe()



,square_meters,num_rooms,distance_to_center,build_year,condition_score,house_price
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03
mean,137.856442,5.440000,24.961246,1987.560000,0.480085,2.522911e+05
std,64.270220,2.879008,14.418473,21.143118,0.290743,1.171049e+05
min,31.019045,1.000000,0.006735,1950.000000,0.000187,3.196264e+04
25%,81.914119,3.000000,13.004729,1969.000000,0.225957,1.638233e+05
50%,139.297623,5.000000,25.025118,1988.000000,0.464340,2.565345e+05
75%,193.750309,8.000000,37.594415,2006.250000,0.733889,3.348133e+05
max,249.937888,10.000000,49.917376,2023.000000,0.997623,1.343089e+06


## Lineare Regression

In [8]:

#! Lineare Regression

X = df_houses.drop('house_price', axis=1)
y = df_houses['house_price']

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Für statsmodels: Konstante hinzufügen
X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

# OLS-Modell anpassen
model = sm.OLS(y_train, X_train_sm).fit()

# Vorhersagen auf Testdaten
y_pred_sm = model.predict(X_test_sm)

# MSE und RMSE berechnen
mse_sm = mean_squared_error(y_test, y_pred_sm)
print(f"MSE (OLS Regression): {mse_sm:,.2f}")
print(f"RMSE (OLS Regression): {math.sqrt(mse_sm):.2f}\n")

# Zusammenfassung des Modells inklusive p-Werten
print(model.summary())

# Interpretation der p-Werte
print("\nInterpretation der p-Werte:")
for var, pval in model.pvalues.items():
    status = "statistisch signifikant" if pval < 0.05 else "nicht signifikant"
    print(f"{var}: p-Wert = {pval:.4f} -> {status}")



MSE (OLS Regression): 265,745,931.23
RMSE (OLS Regression): 16301.72

                            OLS Regression Results                            
Dep. Variable:            house_price   R-squared:                       0.690
Model:                            OLS   Adj. R-squared:                  0.688
Method:                 Least Squares   F-statistic:                     352.6
Date:                Tue, 04 Mar 2025   Prob (F-statistic):          7.55e-199
Time:                        21:51:37   Log-Likelihood:                -10033.
No. Observations:                 800   AIC:                         2.008e+04
Df Residuals:                     794   BIC:                         2.011e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------

Erläuterung und Interpretation der p-Werte

	•	p-Wert < 0,05:

Dies deutet darauf hin, dass der entsprechende Koeffizient statistisch signifikant ist. Mit anderen Worten, es besteht ein hoher Beweis dafür, dass diese unabhängige Variable einen Einfluss auf den Hauspreis hat und der geschätzte Effekt von Null verschieden ist.

	•	p-Wert ≥ 0,05:
	
In diesem Fall ist der Einfluss der Variable statistisch nicht signifikant. Das bedeutet, dass es nicht genügend Evidenz gibt, um anzunehmen, dass diese Variable einen echten Einfluss auf den Hauspreis hat. Solche Variablen können im Modell eventuell reduziert oder genauer untersucht werden.


## Ridge / Lasso / Elastic net

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)

lasso = Lasso(alpha=1000.0)  # etwas höherer alpha, um Effekt zu sehen
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

elastic = ElasticNet(alpha=1000.0, l1_ratio=0.5)
elastic.fit(X_train, y_train)
y_pred_elastic = elastic.predict(X_test)

mse_ridge = mean_squared_error(y_test, y_pred_ridge)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
mse_elastic = mean_squared_error(y_test, y_pred_elastic)

print(f"RMSE Ridge:      {math.sqrt(mse_ridge):,.2f}")
print(f"RMSE Lasso:      {math.sqrt(mse_lasso):,.2f}")
print(f"RMSE ElasticNet: {math.sqrt(mse_elastic):,.2f}")

MSE Ridge:      16,309.72
MSE Lasso:      17,000.27
MSE ElasticNet: 29,350.44
